In [11]:
import pandas as pd
import folium
import ipywidgets as widgets  
from ipywidgets import SelectMultiple, SelectionRangeSlider
from tabulate import tabulate


# Read in the War Data Set
war_data = pd.read_csv(r'data/war_events.csv', usecols=['YEAR_MONTH', 'Date Mapper', 'EVENT_TYPE', 'SUB_EVENT_TYPE', 'ADMIN1', 'LATITUDE', 'LONGITUDE', 'FATALITIES', 'NOTES'])

war_data = war_data[war_data.ADMIN1.notnull()]
# Read in Exports Data and Rename for Merging
mapper = pd.read_pickle(r'data\commodity_mapper.pickle').set_index('Original').to_dict()['New']

exports = pd.read_csv(r'data/commodity_exports.csv', usecols=['Date Mapper', 'Partner', 'Commodity', 'Netweight (kg)', 'Trade Value (US$)'])

exports['Commodity'] = exports['Commodity'].map(mapper)

exports = exports[exports['Partner'] == 'Ukraine']

# Read in Price Data
prices = pd.read_csv(r'data/commodity_prices.csv')

# Convert Dates to DateTime
prices['Date'], exports['Date Mapper'], war_data['Date Mapper'] = pd.to_datetime(prices['Date']), pd.to_datetime(exports['Date Mapper']), pd.to_datetime(war_data['Date Mapper'])


# Prep for Merge
prices['map'] = prices['Date'].astype(str) + prices['Commodity']

exports['map'] = exports['Date Mapper'].astype(str) + exports['Commodity']

# TODO
# ADD how prices and exports changed after events
commodity_data = exports.merge(prices, how='left', on='map')
commodity_data = commodity_data[commodity_data.Commodity_x != 'Drop']
commodity_data = commodity_data[['Date Mapper', 'Partner', 'Commodity_x', 'Netweight (kg)', 'Trade Value (US$)', 'Amount']]
commodity_data.rename(columns={'Commodity_x':'Commodity'}, inplace=True)
commodity_data['Date'] = commodity_data['Date Mapper'].dt.strftime('%b-%y')


In [12]:
color_map = {'2018': '#DD351A',
             '2019': '#4EA244',
             '2020': '#679FD7',
             '2021': '#D7D767',
             '2022': '#C967D7'}

In [15]:
def generate_map(date: list, event_type: list, region: list, commodity: list) -> 'Subsected DataFrame':
    
    try:
        map_data = war_data[(war_data['YEAR_MONTH'].isin(date))
                        &(war_data['EVENT_TYPE'].isin(event_type))
                        &(war_data['ADMIN1'].isin(region))]
        
        table_data = commodity_data[(commodity_data['Commodity'].isin(commodity)) 
                        &(commodity_data['Date'].isin(date))]
    
    # Account for Non-Selection when rendered
    except IndexError:
        return 'No Data For Selected Parameters'

    # Create the Interative Map
    i_map = folium.Map(
            location=[war_data["LATITUDE"].mean(), war_data["LONGITUDE"].mean()-3.5],
            tiles='openstreetmap',
            zoom_start=5.5,
            zoom_control=False,
            scrollWheelZoom=False,
            dragging=False)

    #TODO Make Prettier
    #Create Markers where events happened
    map_data.apply(
        lambda row: folium.CircleMarker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=(row['FATALITIES'] + 2)/5,
            tooltip='<h5>Click here for more info</h5>',
            popup=row['NOTES'],
            stroke=True,
            weight=2,
            color=color_map['20' + row['YEAR_MONTH'][-2:]],
            fill=True,
            fill_color=color_map['20' + row['YEAR_MONTH'][-2:]],
            opacity=0.9,
            fill_opacity=0.3,
            ).add_to(i_map),
        axis=1)
    
    # Group Data for Pivot
    grouped_war_data = map_data.groupby(['YEAR_MONTH', 'EVENT_TYPE', 'ADMIN1']).count().reset_index()
    # Subsect Data
    grouped_war_data = grouped_war_data[grouped_war_data.columns[:4]]
    # Rename Columns
    grouped_war_data.columns = [*grouped_war_data.columns[:-1], 'Count']
    # Display Column
    print(tabulate(grouped_war_data.sort_values(by='Count', ascending=False),headers=[*grouped_war_data.columns], tablefmt='github', showindex=False), '\n')
    print(tabulate(table_data.groupby(by=['Date', 'Commodity']).median().reset_index(), headers=[*table_data.columns], tablefmt='github', showindex=False))
    return i_map
    

# Display Interactive Table and Graph
_ = widgets.interact(
    generate_map, 
    date = SelectMultiple(options=war_data['YEAR_MONTH'].unique()),
    event_type = SelectMultiple(options=sorted(war_data['EVENT_TYPE'].unique())), 
    region = SelectMultiple(options=sorted(war_data['ADMIN1'].unique())),
    commodity = SelectMultiple(options=sorted(commodity_data['Commodity'].unique()))

)



interactive(children=(SelectMultiple(description='date', options=('Jan-18', 'Feb-18', 'Mar-18', 'Apr-18', 'May…